# 下載訓練資料(這邊使用 wiki Data)
## 主要以 pages-articles.xml.bz2 結尾之檔案類型
* 維基資料集: https://zh.wikipedia.org/wiki/Wikipedia:%E6%95%B0%E6%8D%AE%E5%BA%93%E4%B8%8B%E8%BD%BD
* 這邊以　[zhwiki-latest-pages-meta-current.xml.bz2　為訓練集](https://dumps.wikimedia.org/zhwiki/latest/)。

# 將 Wiki 資料轉為 txt(請先下載 zhwiki-latest-pages-meta-current.xml.bz2 至同一個目錄下)

In [ ]:
# pip install gensim

In [1]:
# -*- coding: utf-8 -*-
__author__ = "ALEX-CHUN-YU (P76064538@mail.ncku.edu.tw)"
import logging
import sys
import warnings
warnings.filterwarnings(action ='ignore', category = UserWarning, module = 'gensim')
from gensim.corpora import WikiCorpus

# 將 wiki 資料集載下後進行 xml convert to txt 
class Wiki_to_txt(object):

    def __init__(self):
        # 用默認 Formatter 為日誌系統建立一個 StreamHandler ，設置基礎配置並加到 root logger 中
        logging.basicConfig(format = '%(asctime)s : %(levelname)s : %(message)s', level = logging.INFO)

    # 使用方法 https://radimrehurek.com/gensim/corpora/wikicorpus.html
    def set_wiki_to_txt(self, wiki_data_path = None):
        if wiki_data_path == None:
            # 系統下參數
            if len(sys.argv) != 2:
                print("Please Usage: python3 " + sys.argv[0] + " wiki_data_path")
                exit()
            else:
                wiki_corpus = WikiCorpus(sys.argv[1], dictionary = {})
        else:
            wiki_corpus = WikiCorpus(wiki_data_path, dictionary = {})
        # wiki.xml convert to wiki.txt
        with open("wiki_text.txt", 'w', encoding = 'utf-8') as output:
            text_count = 0
            for text in wiki_corpus.get_texts():
                # save use string(gensim)
                output.write(' '.join(text) + '\n')
                text_count += 1
                if text_count % 10000 == 0:
                    logging.info("目前已處理 %d 篇文章" % text_count)
            print("轉檔完畢!")
            
if __name__ == "__main__":
    wiki_to_txt = Wiki_to_txt()
    # 將 wiki xml 轉換成 wiki txt
    wiki_to_txt.set_wiki_to_txt("zhwiki-latest-pages-meta-current.xml.bz2")


2018-09-15 21:38:57,870 : INFO : 目前已處理 10000 篇文章
2018-09-15 21:39:29,135 : INFO : 目前已處理 20000 篇文章
2018-09-15 21:39:58,811 : INFO : 目前已處理 30000 篇文章
2018-09-15 21:40:32,110 : INFO : 目前已處理 40000 篇文章
2018-09-15 21:41:05,401 : INFO : 目前已處理 50000 篇文章
2018-09-15 21:41:42,561 : INFO : 目前已處理 60000 篇文章
2018-09-15 21:42:14,659 : INFO : 目前已處理 70000 篇文章
2018-09-15 21:42:47,894 : INFO : 目前已處理 80000 篇文章
2018-09-15 21:43:16,987 : INFO : 目前已處理 90000 篇文章
2018-09-15 21:43:47,783 : INFO : 目前已處理 100000 篇文章
2018-09-15 21:44:31,474 : INFO : 目前已處理 110000 篇文章
2018-09-15 21:45:06,963 : INFO : 目前已處理 120000 篇文章
2018-09-15 21:45:43,361 : INFO : 目前已處理 130000 篇文章
2018-09-15 21:46:21,481 : INFO : 目前已處理 140000 篇文章
2018-09-15 21:47:09,781 : INFO : 目前已處理 150000 篇文章
2018-09-15 21:47:48,737 : INFO : 目前已處理 160000 篇文章
2018-09-15 21:48:27,618 : INFO : 目前已處理 170000 篇文章
2018-09-15 21:49:27,423 : INFO : 目前已處理 180000 篇文章
2018-09-15 21:52:04,706 : INFO : 目前已處理 190000 篇文章
2018-09-15 21:53:36,284 : INFO : 目前已處理 200000 篇文章
2018-09-1

轉檔完畢!


# 簡體轉繁體 -> 進行斷詞 -> 過濾[停用詞](https://github.com/Alex-CHUN-YU/Word2vec/blob/master/stopwords.txt)

In [ ]:
# pip install jieba
# pip install hanziconv

In [6]:
# -*- coding: utf-8 -*-
__author__ = "ALEX-CHUN-YU (P76064538@mail.ncku.edu.tw)"
import jieba
import logging
from hanziconv import HanziConv

# 進行斷詞並過濾 stopword
class Segmentation(object):

    def __init__(self):
        # 用默認 Formatter 為日誌系統建立一個 StreamHandler ，設置基礎配置並加到 root logger 中
        logging.basicConfig(format = "%(asctime)s : %(levelname)s : %(message)s", level = logging.INFO)
        self.stopwordset = set()
        
    # 讀取 stopword 辭典，並存到 stopwordset
    def set_stopword(self):
        with open("stopwords.txt", "r", encoding = "utf-8") as stopwords:
            for stopword in stopwords:
                self.stopwordset.add(stopword.strip('\n'))
        #print(self.stopwordset)
        print("StopWord Set 已儲存!")

    # 簡 to 繁
    def simplified_to_traditional(self):
        logging.info("等待中..(簡 to 繁)")
        traditional = open("traditional.txt", "w", encoding = "utf-8")
        with open("wiki_text.txt", "r", encoding = "utf-8") as simplified:
            for s in simplified:
                traditional.write(HanziConv.toTraditional(s))
        print("成功簡體轉繁體!")
        traditional.close()

    # 斷詞(Segmentation)並過濾掉停用詞(Stop Word)
    def segmentation(self):
        logging.info("等待中..(jieba 斷詞，並過濾停用詞)")
        segmentation = open("segmentation.txt", "w", encoding = "utf-8")
        with open("traditional.txt", "r", encoding = "utf-8") as Corpus:
            for sentence in Corpus:
                sentence = sentence.strip("\n")
                pos = jieba.cut(sentence, cut_all = False)
                for term in pos:
                    if term not in self.stopwordset:
                        segmentation.write(term + " ")
        print("jieba 斷詞完畢，並已完成過濾停用詞!")
        segmentation.close()

if __name__ == "__main__":
    segmentation = Segmentation()
    # 讀取停用詞辭典
    segmentation.set_stopword()
    # data 進行簡體轉繁體
    segmentation.simplified_to_traditional()
    # 進行 jieba 斷詞同步過濾停用詞，並產生辭典
    segmentation.segmentation()

2018-09-15 22:20:38,047 : INFO : 等待中..(簡 to 繁)


StopWord Set 已儲存!


2018-09-15 22:25:03,490 : INFO : 等待中..(jieba 斷詞，並過濾停用詞)
Building prefix dict from the default dictionary ...
2018-09-15 22:25:03,491 : DEBUG : Building prefix dict from the default dictionary ...


成功簡體轉繁體!


Dumping model to file cache /var/folders/6k/ms64847j05s5_t0h08zjmwnr0000gp/T/jieba.cache
2018-09-15 22:25:04,149 : DEBUG : Dumping model to file cache /var/folders/6k/ms64847j05s5_t0h08zjmwnr0000gp/T/jieba.cache
Loading model cost 0.725 seconds.
2018-09-15 22:25:04,217 : DEBUG : Loading model cost 0.725 seconds.
Prefix dict has been built succesfully.
2018-09-15 22:25:04,218 : DEBUG : Prefix dict has been built succesfully.


jieba 斷詞完畢，並已完成過濾停用詞!


# 透過 Gensim Word2Vec 來進行訓練(這邊使用 Skip-gram 模型，Dimension 設為 300 維度)

In [7]:
# -*- coding: utf-8 -*-
__author__ = "ALEX-CHUN-YU (P76064538@mail.ncku.edu.tw)"
import warnings
warnings.filterwarnings(action = 'ignore', category = UserWarning, module = 'gensim')
from gensim.models import word2vec

# 主要透過 gensim 訓練成 model 並供使用
class Train(object):

    def __init__(self):
        pass

    # 可參考 https://radimrehurek.com/gensim/models/word2vec.html 更多運用
    def train(self):
        print("訓練中...(喝個咖啡吧^0^)")
        # Load file
        sentence = word2vec.Text8Corpus("segmentation.txt")
        # Setting degree and Produce Model(Train)
        model = word2vec.Word2Vec(sentence, size = 300, window = 10, min_count = 5, workers = 4, sg = 1)
        # Save model 
        model.wv.save_word2vec_format(u"wiki300.model.bin", binary = True)
        print("model 已儲存完畢")

if __name__ == "__main__":
    t = Train()
    # 訓練(shallow semantic space)
    t.train()


2018-09-15 23:28:25,640 : INFO : collecting all words and their counts
2018-09-15 23:28:25,654 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types


訓練中...(喝個咖啡吧^0^)


2018-09-15 23:29:04,301 : INFO : PROGRESS: at sentence #10000, processed 100000000 words, keeping 3909689 word types
2018-09-15 23:29:34,817 : INFO : collected 5457659 word types from a corpus of 171163187 raw words and 17117 sentences
2018-09-15 23:29:34,820 : INFO : Loading a fresh vocabulary
2018-09-15 23:29:39,072 : INFO : effective_min_count=5 retains 1020890 unique words (18% of original 5457659, drops 4436769)
2018-09-15 23:29:39,073 : INFO : effective_min_count=5 leaves 164617681 word corpus (96% of original 171163187, drops 6545506)
2018-09-15 23:29:41,515 : INFO : deleting the raw counts dictionary of 5457659 items
2018-09-15 23:29:41,685 : INFO : sample=0.001 downsamples 16 most-common words
2018-09-15 23:29:41,686 : INFO : downsampling leaves estimated 161870805 word corpus (98.3% of prior 164617681)
2018-09-15 23:29:45,710 : INFO : estimated required memory for 1020890 words and 300 dimensions: 2960581000 bytes
2018-09-15 23:29:45,711 : INFO : resetting layer weights
2018-

2018-09-15 23:31:11,290 : INFO : EPOCH 1 - PROGRESS: at 3.24% examples, 73300 words/s, in_qsize 8, out_qsize 0
2018-09-15 23:31:12,447 : INFO : EPOCH 1 - PROGRESS: at 3.31% examples, 73558 words/s, in_qsize 7, out_qsize 0
2018-09-15 23:31:13,587 : INFO : EPOCH 1 - PROGRESS: at 3.37% examples, 73706 words/s, in_qsize 7, out_qsize 0
2018-09-15 23:31:14,591 : INFO : EPOCH 1 - PROGRESS: at 3.42% examples, 73858 words/s, in_qsize 7, out_qsize 0
2018-09-15 23:31:15,708 : INFO : EPOCH 1 - PROGRESS: at 3.47% examples, 73890 words/s, in_qsize 7, out_qsize 0
2018-09-15 23:31:16,778 : INFO : EPOCH 1 - PROGRESS: at 3.53% examples, 74065 words/s, in_qsize 7, out_qsize 0
2018-09-15 23:31:17,904 : INFO : EPOCH 1 - PROGRESS: at 3.59% examples, 74195 words/s, in_qsize 8, out_qsize 0
2018-09-15 23:31:18,936 : INFO : EPOCH 1 - PROGRESS: at 3.64% examples, 74304 words/s, in_qsize 7, out_qsize 0
2018-09-15 23:31:19,957 : INFO : EPOCH 1 - PROGRESS: at 3.69% examples, 74300 words/s, in_qsize 7, out_qsize 0
2

2018-09-15 23:32:34,917 : INFO : EPOCH 1 - PROGRESS: at 6.94% examples, 72270 words/s, in_qsize 8, out_qsize 0
2018-09-15 23:32:35,939 : INFO : EPOCH 1 - PROGRESS: at 6.98% examples, 72221 words/s, in_qsize 8, out_qsize 0
2018-09-15 23:32:37,009 : INFO : EPOCH 1 - PROGRESS: at 7.02% examples, 72148 words/s, in_qsize 7, out_qsize 0
2018-09-15 23:32:38,053 : INFO : EPOCH 1 - PROGRESS: at 7.08% examples, 72270 words/s, in_qsize 7, out_qsize 0
2018-09-15 23:32:39,116 : INFO : EPOCH 1 - PROGRESS: at 7.12% examples, 72143 words/s, in_qsize 8, out_qsize 0
2018-09-15 23:32:40,146 : INFO : EPOCH 1 - PROGRESS: at 7.17% examples, 72210 words/s, in_qsize 7, out_qsize 0
2018-09-15 23:32:41,252 : INFO : EPOCH 1 - PROGRESS: at 7.21% examples, 72125 words/s, in_qsize 7, out_qsize 0
2018-09-15 23:32:42,272 : INFO : EPOCH 1 - PROGRESS: at 7.26% examples, 72196 words/s, in_qsize 8, out_qsize 0
2018-09-15 23:32:43,312 : INFO : EPOCH 1 - PROGRESS: at 7.31% examples, 72195 words/s, in_qsize 7, out_qsize 0
2

...

2018-09-16 02:21:02,695 : INFO : EPOCH 5 - PROGRESS: at 88.07% examples, 77488 words/s, in_qsize 7, out_qsize 0
2018-09-16 02:21:03,701 : INFO : EPOCH 5 - PROGRESS: at 88.12% examples, 77487 words/s, in_qsize 6, out_qsize 1
2018-09-16 02:21:04,880 : INFO : EPOCH 5 - PROGRESS: at 88.18% examples, 77493 words/s, in_qsize 8, out_qsize 0
2018-09-16 02:21:06,022 : INFO : EPOCH 5 - PROGRESS: at 88.23% examples, 77492 words/s, in_qsize 8, out_qsize 0
2018-09-16 02:21:07,065 : INFO : EPOCH 5 - PROGRESS: at 88.28% examples, 77489 words/s, in_qsize 7, out_qsize 0
2018-09-16 02:21:08,093 : INFO : EPOCH 5 - PROGRESS: at 88.34% examples, 77497 words/s, in_qsize 7, out_qsize 0
2018-09-16 02:21:09,160 : INFO : EPOCH 5 - PROGRESS: at 88.38% examples, 77488 words/s, in_qsize 7, out_qsize 0
2018-09-16 02:21:10,316 : INFO : EPOCH 5 - PROGRESS: at 88.44% examples, 77491 words/s, in_qsize 7, out_qsize 0
2018-09-16 02:21:11,511 : INFO : EPOCH 5 - PROGRESS: at 88.49% examples, 77488 words/s, in_qsize 7, out_

2018-09-16 02:22:22,852 : INFO : EPOCH 5 - PROGRESS: at 91.86% examples, 77465 words/s, in_qsize 7, out_qsize 0
2018-09-16 02:22:23,932 : INFO : EPOCH 5 - PROGRESS: at 91.91% examples, 77466 words/s, in_qsize 7, out_qsize 0
2018-09-16 02:22:24,974 : INFO : EPOCH 5 - PROGRESS: at 91.96% examples, 77464 words/s, in_qsize 7, out_qsize 0
2018-09-16 02:22:25,995 : INFO : EPOCH 5 - PROGRESS: at 92.01% examples, 77462 words/s, in_qsize 7, out_qsize 0
2018-09-16 02:22:27,292 : INFO : EPOCH 5 - PROGRESS: at 92.06% examples, 77454 words/s, in_qsize 7, out_qsize 0
2018-09-16 02:22:28,440 : INFO : EPOCH 5 - PROGRESS: at 92.12% examples, 77462 words/s, in_qsize 7, out_qsize 0
2018-09-16 02:22:29,627 : INFO : EPOCH 5 - PROGRESS: at 92.18% examples, 77459 words/s, in_qsize 8, out_qsize 0
2018-09-16 02:22:30,780 : INFO : EPOCH 5 - PROGRESS: at 92.22% examples, 77452 words/s, in_qsize 7, out_qsize 0
2018-09-16 02:22:31,886 : INFO : EPOCH 5 - PROGRESS: at 92.29% examples, 77462 words/s, in_qsize 7, out_

2018-09-16 02:23:43,259 : INFO : EPOCH 5 - PROGRESS: at 95.65% examples, 77425 words/s, in_qsize 8, out_qsize 0
2018-09-16 02:23:44,478 : INFO : EPOCH 5 - PROGRESS: at 95.71% examples, 77425 words/s, in_qsize 7, out_qsize 0
2018-09-16 02:23:45,586 : INFO : EPOCH 5 - PROGRESS: at 95.76% examples, 77420 words/s, in_qsize 7, out_qsize 0
2018-09-16 02:23:46,586 : INFO : EPOCH 5 - PROGRESS: at 95.81% examples, 77424 words/s, in_qsize 7, out_qsize 0
2018-09-16 02:23:47,721 : INFO : EPOCH 5 - PROGRESS: at 95.86% examples, 77423 words/s, in_qsize 8, out_qsize 0
2018-09-16 02:23:48,725 : INFO : EPOCH 5 - PROGRESS: at 95.91% examples, 77422 words/s, in_qsize 7, out_qsize 0
2018-09-16 02:23:49,917 : INFO : EPOCH 5 - PROGRESS: at 95.97% examples, 77423 words/s, in_qsize 7, out_qsize 0
2018-09-16 02:23:50,950 : INFO : EPOCH 5 - PROGRESS: at 96.02% examples, 77421 words/s, in_qsize 7, out_qsize 0
2018-09-16 02:23:51,963 : INFO : EPOCH 5 - PROGRESS: at 96.06% examples, 77420 words/s, in_qsize 7, out_

2018-09-16 02:25:04,064 : INFO : EPOCH 5 - PROGRESS: at 99.47% examples, 77392 words/s, in_qsize 7, out_qsize 0
2018-09-16 02:25:05,122 : INFO : EPOCH 5 - PROGRESS: at 99.52% examples, 77389 words/s, in_qsize 7, out_qsize 0
2018-09-16 02:25:06,263 : INFO : EPOCH 5 - PROGRESS: at 99.57% examples, 77387 words/s, in_qsize 7, out_qsize 0
2018-09-16 02:25:07,372 : INFO : EPOCH 5 - PROGRESS: at 99.63% examples, 77391 words/s, in_qsize 7, out_qsize 0
2018-09-16 02:25:08,408 : INFO : EPOCH 5 - PROGRESS: at 99.68% examples, 77394 words/s, in_qsize 7, out_qsize 0
2018-09-16 02:25:09,461 : INFO : EPOCH 5 - PROGRESS: at 99.73% examples, 77391 words/s, in_qsize 7, out_qsize 0
2018-09-16 02:25:10,522 : INFO : EPOCH 5 - PROGRESS: at 99.78% examples, 77388 words/s, in_qsize 7, out_qsize 0
2018-09-16 02:25:11,625 : INFO : EPOCH 5 - PROGRESS: at 99.83% examples, 77387 words/s, in_qsize 7, out_qsize 0
2018-09-16 02:25:12,760 : INFO : EPOCH 5 - PROGRESS: at 99.89% examples, 77390 words/s, in_qsize 7, out_

model 已儲存完畢


# 將訓練好的 Model 進行測試

In [8]:
# -*- coding: utf-8 -*-
__author__ = "ALEX-CHUN-YU (P76064538@mail.ncku.edu.tw)"
import warnings
warnings.filterwarnings(action = 'ignore', category = UserWarning, module = 'gensim')
from gensim.models.keyedvectors import KeyedVectors

# 載入 model 並去運用
def main():
    # 可參考 https://radimrehurek.com/gensim/models/word2vec.html 更多運用
    # How to use bin(model)?
    word_vectors = KeyedVectors.load_word2vec_format("wiki300.model.bin", binary = True)
    print("'爸爸'前10名相似:")    
    res = word_vectors.wv.most_similar('爸爸', topn = 10)
    for item in res:
        print(item[0] + "," + str(item[1]))
    print("\n'爸爸','媽媽'之間相似度:")
    res = word_vectors.similarity('爸爸', '媽媽')
    print(res)
    print("\n'爸爸'之於'老公',如'媽媽'之於'老婆':")
    res = word_vectors.most_similar(positive = ['爸爸', '老公'], negative = ['媽媽'], topn = 5)
    for item in res:
        print(item[0] + "," + str(item[1]))

if __name__ == "__main__":
    main()


2018-09-16 11:12:44,386 : INFO : loading projection weights from wiki300.model.bin
2018-09-16 11:12:55,521 : INFO : loaded (1020890, 300) matrix from wiki300.model.bin
/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  del sys.path[0]
2018-09-16 11:12:55,529 : INFO : precomputing L2-norms of word weight vectors


'爸爸'前10名相似:


/anaconda3/envs/tensorflow/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


媽媽,0.853445291519165
奶奶,0.7114385366439819
爺爺,0.700326144695282
哥哥,0.670133113861084
賢彬,0.6615862250328064
老爸,0.6546416282653809
老婆,0.6543856859207153
慧娜,0.6535881161689758
仁好,0.6523171067237854
老公,0.6493898034095764

'爸爸','媽媽'之間相似度:
0.0

'爸爸'之於'老公',如'媽媽'之於'老婆':
老婆,0.63941890001297
梁九妹,0.5862223505973816
男友,0.5829707384109497
發財夢,0.578301191329956
夠秤,0.5750516653060913
